In [1]:
using IterativeSolvers
using LinearAlgebra
using LinearMaps
using SparseArrays

In [2]:
function read_mat()
    Is = Int64[]
    Js = Int64[]
    # data downloaded from http://snap.stanford.edu/data/enwiki-2013.html
    open("enwiki-2013.txt") do f
        for line in eachline(f)
            if line[1] == '#'; continue; end # skip lines starting with #
            i, j = split(line)
            # note 0 --> 1 indexing
            push!(Is, parse(Int64, i) + 1)
            push!(Js, parse(Int64, j) + 1)
        end
    end
    n = max(maximum(Is), maximum(Js))
    A = sparse(Is, Js, 1, n, n)
    A = min.(A, 1)
    A = max.(A, A')  # symmetrize
    d = vec(sum(A, dims=1))
    # Skip unused indices
    keep = d .> 0
    return A[keep, keep]
end

read_mat (generic function with 1 method)

In [3]:
@time A = read_mat()  # takes about one minute
@time A * randn(size(A,1))  # takes about a second
;

 69.081940 seconds (405.49 M allocations: 35.578 GiB, 4.64% gc time)
  1.247815 seconds (216.46 k allocations: 75.669 MiB, 32.09% gc time)


In [4]:
d = vec(sum(A, dims=1))
D = Diagonal(d)
L = Float64.(D - A)  # Graph Laplacian
n = size(L, 1)
b = randn(n)  # right-hand-side
x = randn(n)  # starting guess
@show norm(L * x - b) / norm(b)  # starting relative error
cg!(x, L, b, maxiter=20)
@show norm(L * x - b) / norm(b)  # relative error after some CG steps
;

norm(L * x - b) / norm(b) = 441.7198599247834
norm(L * x - b) / norm(b) = 163.96819375433833


In [5]:
v = randn(n)
# (L + vv^T) * x
L_plus_vvt = LinearMap(x -> L * x .+ v .* (v' * x), n, 
                       issymmetric=true, ismutating=false)
@show norm(L_plus_vvt * x - b) / norm(b)
cg!(x, L_plus_vvt, b, maxiter=20)
@show norm(L_plus_vvt * x - b) / norm(b)

norm(L_plus_vvt * x - b) / norm(b) = 168.36768330320803
norm(L_plus_vvt * x - b) / norm(b) = 133.7621181612782


133.7621181612782